In [1]:
# Visualisation functions
function firing_rates(t_grid, θ::PoissonProcessParams, KernList::AbstractArray; n_range=1, d_range=1, m_range=1)
    C, α, u, N, M, D, J = name_params(θ)
    b = θ.b
    latent_funcs = Array(Any,(maximum(n_range), maximum(d_range)))
    for n = n_range
        for d = d_range
            tmp = function(t::Float64)
                out = 0
                for j = 1:length(u[d])
                     out += (α[n,d][j] * kernel(KernList[d], t, u[d][j]));
                end
                return out
            end
            latent_funcs[n, d] = tmp
        end
    end
    
    firing_rate = Array(Any, (maximum(n_range), maximum(m_range)))
    for n = n_range
        for m = m_range
            firing_rate[n,m] = zeros(length(t_grid))
            for t = 1:length(t_grid)
                for d = d_range
                    firing_rate[n,m][t] += C[m,d]*latent_funcs[n,d](t_grid[t]);
                end
            end
            firing_rate[n,m] = exp(firing_rate[n,m]+b[m]) # exponential link function
        end
    end
                
    return firing_rate
end

firing_rates (generic function with 1 method)

In [ ]:
# Plot data, true rate function and estimated rate function for a neuron on given trials
using PlotlyJS

function plot_params(data, θ_opt, θ_orig, KernList; 
                     num_neur = 1, trials = 1:5, dim_latent = [])
    
    C, α, u, N, M, D, J = name_params(θ_opt)
    if isempty(dim_latent)
        dim_latent = 1:D
    end

    #Plot estimated
    to_plot = Array(PlotlyJS.GenericTrace{Dict{Symbol,Any}}, size(data,1))
    rate_rand_plot = Array(PlotlyJS.GenericTrace{Dict{Symbol,Any}}, size(data,1))
    rate_est_plot = Array(PlotlyJS.GenericTrace{Dict{Symbol,Any}}, size(data,1))
    latent_plot = Array(PlotlyJS.GenericTrace{Dict{Symbol,Any}}, size(data,1))
    latent_est_plot = Array(PlotlyJS.GenericTrace{Dict{Symbol,Any}}, size(data,1))
    colors = ["rgb(0,0,255)", "rgb(0,255,0)", "rgb(255,0,0)", "rgb(128,128,0)", "rgb(0,128,128)"]
    for i1 = trials
        #@show collect(data[i1][num_neur])
        # Plot data
        to_plot[i1] = scatter(;x=collect(data[i1][num_neur]), y=i1*collect(ones(size(data[i1][num_neur]))), mode="markers", marker_color=colors[mod(i1,5)+1], yaxis="y2")

        # Compute underlying rate and latent functions
        rate_rand = firing_rates(0:0.001:1, θ_orig, KernList, n_range=i1, d_range=dim_latent, m_range=num_neur)
        rate_est = firing_rates(0:0.001:1, θ_opt, KernList, n_range=i1, d_range=dim_latent, m_range=num_neur)


        rate_est_plot[i1] = scatter(; 
        x=collect(0:0.001:1), 
        y=collect(rate_est[i1, num_neur]),
        line_color = colors[mod(i1,5)+1],
        yaxis = "y0"
    )

        rate_rand_plot[i1] = scatter(; 
        x=collect(0:0.001:1), 
        y=collect(rate_rand[i1, num_neur]),
        line_color = colors[mod(i1,5)+1],
        yaxis = "y0"
    )

    end

    lo = Layout(;xaxis_range=[0,1])

    #plt = plot([Plot(to_plot[trials]); Plot(rate_plot[trials]);  Plot(rate_est_plot[trials]);  Plot(latent_plot[trials]);  Plot(latent_est_plot[trials])])
    #plt = plot([Plot(to_plot[trials], lo); Plot(rate_est_plot[trials], lo); Plot(rate_rand_plot[trials],lo)])
    plt = plot([Plot(to_plot[trials], lo); Plot(rate_est_plot[trials],lo); Plot(rate_rand_plot[trials],lo)])

    relayout!(plt, height=700)

    plt
end